### 아래 내용은 글로벌 기업 (주)인텔에서 개발한 <br> Intel® AI for Youth Program 내용을 
대한민국 Implementation Partner사인 (주)Brain AI와 Brain AI Lead Coach Network에서 우리나라 초, 중, 고 학생들의 AI 교육을 위해 현지화 한 내용입니다. <br>
(주)Brain AI와 NDA를 체결한 학교에서만 사용 가능하며 NDA를 준수해야합니다. <br>
상업적 사용은 불가하며, 학교내에서 학생들 교육활동에 자유롭게 사용가능합니다.

# 프로젝트 제목: Brain AI 감정 인식 AI 시스템 만들기
Intel OpenVINO Pre-trained 모델을 활용하여 감정 인식 AI 시스템 개발 실습

## 1.  Compile Model  

In [2]:
import os

from flask_wtf import FlaskForm
from wtforms.fields import FileField, SubmitField
from wtforms.validators import InputRequired
from openvino.runtime import Core
from flask import Flask, render_template, request


In [3]:
ie = Core()

model = ie.read_model(model="./model/face-detection-adas-0001.xml")
face_model = ie.compile_model(model=model, device_name="CPU")

face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)
print("Input layer shape: ", face_input_layer.shape)
print("Output layer shape:",face_output_layer.shape)

Input layer shape:  [1,3,384,672]
Output layer shape: [1,1,200,7]


In [4]:
model = ie.read_model(model="./model/emotions-recognition-retail-0003.xml")
emotion_model = ie.compile_model(model=model, device_name="CPU")

emotion_input_layer = model.input(0)
emotion_output_layer = emotion_model.output(0)
print("Input layer shape: ", emotion_input_layer.shape)
print("Output layer shape:",emotion_output_layer.shape)

Input layer shape:  [1,3,64,64]
Output layer shape: [1,5,1,1]


## 2. 새로운 데이터 준비

In [5]:
import cv2
import numpy as np

# frame = cv2.imread("images/emotions.jpg")

# resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
# transposed_frame = resized_frame.transpose(2, 0, 1)
# input_frame = np.expand_dims(transposed_frame, 0)

## 3. 추론 및 배포 Inference + Deployment

In [6]:
def DrawBoundingBoxes(output, frame, conf=0.5):
    boxes = []
    frame.copy()                             # 원본 이미지를 수정하는 대신 복사합니다
    h,w,_ = frame.shape
    
    predictions = output[0][0]                         # 하위 집합 데이터 프레임
    confidence = predictions[:,2]                       # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]
  
    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택
 
    for detection in top_predictions:
        box = (detection[3:7]* np.array([w, h, w, h])).astype("int") # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box   # xmin, ymin, xmax, ymax에 상자 위치 값 지정
        boxes.append(box)
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 만들기
    
    return boxes

In [7]:
def DrawText(output, frame, face_position):
    
    emotions = {
        0:"neutral",
        1:"happy",
        2:"sad",
        3:"surprise",
        4:"anger"
    }
                
    predictions = output[0,:,0,0]
   
    topresult_index = np.argmax(predictions)
    emotion = emotions[topresult_index]
   
    cv2.putText(frame, emotion,
                (face_position[0],face_position[1]),
                cv2.FONT_HERSHEY_SIMPLEX, 2, 
                (255, 255,255), 4)

In [10]:
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()

while webcam.isOpened():
    status, frame = webcam.read()

    if status:
        resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
        transposed_frame = resized_frame.transpose(2, 0, 1)
        input_frame = np.expand_dims(transposed_frame, 0)
    
        face_output = face_model([input_frame])[face_output_layer]
        boxes = DrawBoundingBoxes(face_output, frame, conf=0.5)
        
        if boxes is not None:
            
            for box in boxes:
            
                xmin, ymin, xmax, ymax = box
                emotion_input = frame[ymin:ymax,xmin:xmax]
                if emotion_input.shape[0] < 64 or emotion_input.shape[1] < 64: continue
                
                resized_image = cv2.resize(src=emotion_input, dsize=(64, 64))
                transposed_image = resized_image.transpose(2, 0, 1)
                input_image = np.expand_dims(transposed_image, 0)
        
                emotion_output = emotion_model([input_image])[emotion_output_layer]
                DrawText(emotion_output, frame, box)
        
        cv2.imshow("Face", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()

cv2.waitKey(0)
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 

In [8]:
app = Flask(__name__)

app.config["SECRET_KEY"] = ""

class UploadFileForm(FlaskForm):
    file = FileField("File", validators = [ InputRequired() ])
    submit = SubmitField("Upload File")

@app.route("/", methods=["GET"])
def index():
    return render_template('index.html')

@app.route("/recognize", methods=["POST"])
def recognize():
    form = UploadFileForm()
    
    if form.validate_on_submit():
        file = form.file.data
        file_type = str(os.path.splitext(file.filename)[1])

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Apr/2024 17:39:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2024 17:39:11] "GET /static/styles/index-plus.css HTTP/1.1" 304 -
127.0.0.1 - - [15/Apr/2024 17:39:11] "GET /static/styles/index.css HTTP/1.1" 304 -
127.0.0.1 - - [15/Apr/2024 17:39:11] "GET /static/scripts/index.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Apr/2024 17:39:11] "GET /static/audio/happy.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [15/Apr/2024 17:39:11] "GET /static/audio/neutral.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [15/Apr/2024 17:39:11] "GET /static/audio/sad.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [15/Apr/2024 17:39:11] "GET /static/audio/surprise.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [15/Apr/2024 17:39:11] "GET /static/audio/anger.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [15/Apr/2024 17:40:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2024 17:40:12] "GET /static/scripts/index.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Apr/2024 17:40:12] "GET /static/styles/index-plus.css H